In [55]:
from datetime import datetime
import pandas as pd 
import ast

# Reading the .csv with just the tweets published in 2019 

In [56]:
tweets = pd.read_csv('2019_tweets.csv', lineterminator='\n', index_col=0)

## Converting the string_date into datetime

In [57]:
tweets['created_at'] = pd.to_datetime(tweets['created_at'], errors='coerce').dt.date

In [58]:
daily_stats = tweets.groupby(['user_id', 'created_at'], as_index=False).sum()
daily_stats['acceptance'] = daily_stats.retweet_count + daily_stats.reply_count + daily_stats.favorite_count
daily_stats['diffusion'] = daily_stats.num_hashtags + daily_stats.num_mentions + daily_stats.num_urls + 0.1
daily_stats['success'] = daily_stats.acceptance / daily_stats.diffusion

/tmp/ipykernel_19713/70603973.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily_stats = tweets.groupby(['user_id', 'created_at'], as_index=False).sum()


## Calculating the success ratio

In [59]:
daily_stats['success'] = round(daily_stats['success'], 2)
daily_stats = daily_stats[daily_stats['user_id'] > 0.0]

## Extracting ordinal numbers from datetime

In [60]:
daily_stats['ordinal'] = daily_stats['created_at'].apply(datetime.toordinal) - 737060
daily_stats = daily_stats[['user_id', 'ordinal', 'success']]

In [61]:
MAX = daily_stats['ordinal'].max()
MAX

318

In [62]:
len(daily_stats.index)

518090

In [63]:
daily_stats = daily_stats.groupby(['user_id', 'ordinal'], as_index=False).sum()

In [64]:
len(daily_stats.index)

518090

## Aggregating datas into a list

In [65]:
daily_stats['ts'] = list(zip(daily_stats.ordinal, daily_stats.success))
daily_stats = daily_stats.groupby('user_id')['ts'].apply(lambda x : x.values.tolist())

In [66]:
d = daily_stats.to_dict()
nd = {}
for u in d:
    nd[u] = {}
    for couple in d[u]:
        nd[u][couple[0]] = couple[1]
d = nd
d

{3.0: {159: 0.0},
 5.0: {173: 0.0},
 6.0: {291: 0.0},
 8.0: {277: 0.0},
 9.0: {284: 0.0},
 17.0: {297: 0.0},
 20.0: {292: 0.48},
 31.0: {306: 0.0},
 33.0: {174: 0.0},
 35.0: {292: 120.0},
 40.0: {139: 40.0},
 50.0: {294: 0.0},
 58.0: {118: 0.0},
 62.0: {238: 0.0},
 63.0: {233: 10.0, 305: 0.91},
 64.0: {236: 0.0},
 65.0: {67: 0.0, 122: 0.0},
 70.0: {12: 0.0, 211: 0.0},
 73.0: {129: 0.0, 256: 0.0},
 86.0: {279: 0.0},
 89.0: {290: 0.0},
 101.0: {203: 0.0},
 102.0: {217: 15.0, 270: 0.0},
 104.0: {121: 1.19, 179: 20.0},
 108.0: {271: 0.0},
 117.0: {204: 10.0},
 118.0: {264: 0.0},
 128.0: {157: 0.0, 305: 0.0},
 129.0: {195: 0.91},
 135.0: {175: 0.0},
 139.0: {316: 20.0},
 142.0: {303: 0.0},
 143.0: {280: 0.0},
 144.0: {99: 0.0, 189: 0.0, 292: 0.0},
 149.0: {65: 0.0},
 154.0: {305: 0.0},
 158.0: {120: 0.0, 173: 0.91, 262: 0.0},
 162.0: {203: 0.0, 259: 0.0},
 165.0: {137: 1.15, 243: 0.0},
 167.0: {285: 0.0},
 174.0: {214: 0.0},
 175.0: {121: 0.0},
 181.0: {119: 0.0, 293: 0.0},
 183.0: {30: 0.0

### Completing the list

In [67]:
nd = {}
for u in d:
    nd[u] = []
    for i in range (0,MAX+1):
        if i not in d[u]:
            nd[u].append(-1)
        else:
            nd[u].append(d[u][i])

In [68]:
new_stats = pd.DataFrame(nd.items(), columns=['user_id', 'success_ts'])

In [69]:
len(new_stats.index)

7168

## Left joining the bot column

In [70]:
users = pd.read_csv('new_users.csv', lineterminator='\n', index_col=0)

In [71]:
users = users[['user_id', 'bot']]
users['user_id'] = users['user_id'].astype(int)
new_stats['user_id'] = new_stats['user_id'].astype(int)

In [72]:
new_frame = new_stats.merge(users, how='left', on='user_id')

In [73]:
new_frame = new_frame.sort_values('user_id')

In [74]:
new_frame = new_frame[new_frame['user_id'] > 0]

# Write in a new .csv

In [75]:
new_frame.to_csv('daily_stats_user.csv')